# Streaming audio data

Ses veri kümelerinde karşılaşılan en büyük zorluklardan biri büyüklükleridir. Tek bir dakikalık sıkıştırılmamış CD kalitesinde ses (44.1kHz, 16-bit) 5 MB'tan biraz daha fazla depolama alanı kaplar. Tipik olarak, bir ses veri kümesi saatler süren kayıtlar içerir.

Önceki bölümlerde MINDS-14 ses veri kümesinin çok küçük bir alt kümesini kullandık, ancak tipik ses veri kümeleri çok daha büyüktür. Örneğin, SpeechColab'dan GigaSpeech'in xs (en küçük) yapılandırması yalnızca 10 saatlik eğitim verisi içerir, ancak indirme ve hazırlama için 13 GB'tan fazla depolama alanı kaplar. Peki daha büyük bir bölünme üzerinde eğitim yapmak istediğimizde ne olur? Aynı veri setinin tam xl yapılandırması 10.000 saatlik eğitim verisi içerir ve 1 TB'ın üzerinde depolama alanı gerektirir. Çoğumuz için bu, tipik bir sabit disk sürücüsünün özelliklerini oldukça aşıyor. Paraya kıyıp ek depolama alanı satın almamız gerekiyor mu? Yoksa disk alanı kısıtlaması olmadan bu veri kümeleri üzerinde eğitim yapmanın bir yolu var mı?

Datasets, streaming modunu sunarak imdadımıza yetişiyor. Streaming, veri kümesi üzerinde yineleme yaparken verileri aşamalı olarak yüklememize olanak tanır. Tüm veri kümesini bir kerede indirmek yerine, veri kümesini her seferinde bir örnek yükleriz. Veri kümesi üzerinde yineleme yaparak, ihtiyaç duyulduğunda örnekleri anında yükler ve hazırlarız. Bu şekilde, yalnızca kullandığımız örnekleri yüklüyoruz, kullanmadıklarımızı değil! Bir örnekle işimiz bittiğinde, veri kümesi üzerinde yinelemeye devam eder ve bir sonrakini yükleriz.

Streaming modunun tüm veri setini bir kerede indirmeye göre üç temel avantajı vardır:

- Disk alanı: Veri kümesi üzerinde yineleme yaparken örnekler belleğe teker teker yüklenir. Veriler yerel olarak indirilmediğinden, disk alanı gereksinimi yoktur, bu nedenle isteğe bağlı boyuttaki veri kümelerini kullanabilirsiniz.

- İndirme ve işleme süresi: ses veri kümeleri büyüktür ve indirilmesi ve işlenmesi için önemli miktarda zaman gerekir. Akış ile yükleme ve işleme anında yapılır, yani ilk örnek hazır olur olmaz veri kümesini kullanmaya başlayabilirsiniz.

- Kolay deneme: tüm veri setini indirmek zorunda kalmadan kodunuzun çalışıp çalışmadığını kontrol etmek için birkaç örnek üzerinde deneme yapabilirsiniz.

Streaming modu için bir uyarı var. Streaming olmadan tam bir veri kümesi indirilirken, hem ham veriler hem de işlenmiş veriler yerel olarak diske kaydedilir. Bu veri setini yeniden kullanmak istersek, indirme ve işleme adımlarını atlayarak işlenmiş verileri doğrudan diskten yükleyebiliriz. Sonuç olarak, indirme ve işleme işlemlerini yalnızca bir kez gerçekleştirmemiz gerekir, ardından hazırlanan verileri yeniden kullanabiliriz.

Akış modunda veriler diske indirilmez. Böylece ne indirilen ne de önceden işlenen veriler önbelleğe alınmaz. Veri setini yeniden kullanmak istersek, ses dosyaları tekrar yüklenerek ve anında işlenerek akış adımları tekrarlanmalıdır. Bu nedenle, birden çok kez kullanmanız muhtemel olan veri kümelerini indirmeniz önerilir.

Streaming modunu nasıl etkinleştirebilirsiniz? Çok kolay! Veri setinizi yüklediğinizde streaming=True olarak ayarlamanız yeterlidir. Gerisi sizin için halledilecektir:

In [1]:
from huggingface_hub import login

login()

In [2]:
from datasets import load_dataset

gigaspeech = load_dataset(
    "speechcolab/gigaspeech", "xs", 
    streaming=True, trust_remote_code=True
)
gigaspeech

IterableDatasetDict({
    train: IterableDataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        n_shards: 1
    })
    validation: IterableDataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        n_shards: 1
    })
    test: IterableDataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        n_shards: 3
    })
})

Tıpkı MINDS-14'ün indirilmiş bir alt kümesine ön işleme adımlarını uyguladığımız gibi, aynı ön işlemeyi bir akış veri kümesi ile tamamen aynı şekilde yapabilirsiniz. 

Tek fark, artık Python indeksleme kullanarak tek tek örneklere erişememenizdir (yani gigaspeech["train"][sample_idx]). Bunun yerine, veri kümesi üzerinde yineleme yapmanız gerekir. Bir veri kümesini streaming uygularken bir örneğe nasıl erişebileceğiniz aşağıda açıklanmıştır:

In [3]:
next(iter(gigaspeech["train"]))

{'segment_id': 'YOU0000000315_S0000660',
 'speaker': 'N/A',
 'text': "AS THEY'RE LEAVING <COMMA> CAN KASH PULL ZAHRA ASIDE REALLY QUICKLY <QUESTIONMARK>",
 'audio': {'path': 'xs_chunks_0000/YOU0000000315_S0000660.wav',
  'array': array([0.0005188 , 0.00085449, 0.00012207, ..., 0.00125122, 0.00076294,
         0.00036621]),
  'sampling_rate': 16000},
 'begin_time': 2941.89,
 'end_time': 2945.07,
 'audio_id': 'YOU0000000315',
 'title': 'Return to Vasselheim | Critical Role: VOX MACHINA | Episode 43',
 'url': 'https://www.youtube.com/watch?v=zr2n1fLVasU',
 'source': 2,
 'category': 24,
 'original_full_path': 'audio/youtube/P0004/YOU0000000315.opus'}

If you’d like to preview several examples from a large dataset, use the **take()** to get the first n elements. Let’s grab the first two examples in the gigaspeech dataset:

In [4]:
gigaspeech_head = gigaspeech["train"].take(2)
list(gigaspeech_head)

[{'segment_id': 'YOU0000000315_S0000660',
  'speaker': 'N/A',
  'text': "AS THEY'RE LEAVING <COMMA> CAN KASH PULL ZAHRA ASIDE REALLY QUICKLY <QUESTIONMARK>",
  'audio': {'path': 'xs_chunks_0000/YOU0000000315_S0000660.wav',
   'array': array([0.0005188 , 0.00085449, 0.00012207, ..., 0.00125122, 0.00076294,
          0.00036621]),
   'sampling_rate': 16000},
  'begin_time': 2941.89,
  'end_time': 2945.07,
  'audio_id': 'YOU0000000315',
  'title': 'Return to Vasselheim | Critical Role: VOX MACHINA | Episode 43',
  'url': 'https://www.youtube.com/watch?v=zr2n1fLVasU',
  'source': 2,
  'category': 24,
  'original_full_path': 'audio/youtube/P0004/YOU0000000315.opus'},
 {'segment_id': 'AUD0000001043_S0000775',
  'speaker': 'N/A',
  'text': 'SIX TOMATOES <PERIOD>',
  'audio': {'path': 'xs_chunks_0000/AUD0000001043_S0000775.wav',
   'array': array([ 1.43432617e-03,  1.37329102e-03,  1.31225586e-03, ...,
          -6.10351562e-05, -1.22070312e-04, -1.83105469e-04]),
   'sampling_rate': 16000},
 

Akış modu, araştırmanızı bir sonraki seviyeye taşıyabilir: yalnızca en büyük veri kümelerine erişmenizi sağlamakla kalmaz, aynı zamanda birden fazla veri kümesindeki sistemleri, disk alanınız konusunda endişelenmeden tek seferde kolayca değerlendirebilirsiniz. Tek bir veri kümesi üzerinde değerlendirmeyle karşılaştırıldığında, çoklu veri kümesi değerlendirmesi, bir konuşma tanıma sisteminin genelleme yetenekleri için daha iyi bir ölçüm sağlar (bkz. End-to-end Speech Benchmark (ESB)).